# Getty Query Tutorial (Gender)
In this notebook, we perform the similar hunger-for-knowledge procedure for gender in Wikidata and Getty Vocabulary. The main difference is that we generalize the query for single property to the property chain, this is, the path in kgtk.

In [1]:
import os
import json
import pandas as pd

from kgtk.functions import kgtk, kypher

## Step 0: Set up environment paths
Here we set needed Graph files and desired output files to environment paths.

In [2]:
# We will define environment variables to hold the full paths to the files as we will use them in the shell commands
kgtk_environment_variables = []

# Folder where database files store
data_path = "/nas/home/bohuizha/KG/hunger-for-knowledge/data/"
os.environ['DATABASE'] = data_path
kgtk_environment_variables.append('DATABASE')

# Wikidata (all is much less than claims)
os.environ['WIKIDATA'] = data_path + "claims.tsv"
kgtk_environment_variables.append('WIKIDATA')

# Label file of Wikidata
os.environ['KGTK_LABEL_FILE'] = data_path + "labels.en.tsv"
kgtk_environment_variables.append('KGTK_LABEL_FILE')

# P31
os.environ['P31'] = data_path + "P31.tsv"
kgtk_environment_variables.append('P31')

# P279star
os.environ['P279STAR'] = data_path + "P279star.tsv"
kgtk_environment_variables.append('P279STAR')

# Folder of ULAN
ulan_path = data_path + "gvp/ULAN/"
os.environ['ULAN'] = ulan_path
kgtk_environment_variables.append('ULAN')

# File concatenated by used explicit files: ULAN Subjects, AgentMap, Biographies, AAT-Wikidata Alignment
# The suffix ID means we performed add-id operation to the graph to meet the requirement of paths command.
ulan_concat_path = ulan_path + "ulan.gender.concat.id.tsv"
os.environ['ULAN_GENDER_CONCAT_ID'] = ulan_concat_path
kgtk_environment_variables.append('ULAN_GENDER_CONCAT_ID')

# ULAN-Wikidata alignment file, maps ULAN ID to Wikidata Qnode
ulan_wikialign_path = ulan_path + "wiki.align.tsv"
os.environ['ULAN_ALIGN'] = ulan_wikialign_path
kgtk_environment_variables.append('ULAN_ALIGN')

# Output
output_path = data_path + "gvp/gender_output/"
if not os.path.exists(output_path):
    os.mkdir(output_path)
os.environ['OUTPUT'] = output_path
kgtk_environment_variables.append('OUTPUT')

# Each file will be explained later in the procedure
output_names = {
    "wiki_gender": "wiki.gender.tsv",
    "wiki_unknown": "wiki.unknown.tsv",
    "pairs": "pairs.tsv",
    "paths": "paths.tsv",
    "paths_label": "paths.label.tsv",
    "getty_gender": "getty.gender.tsv",
    "getty_mapped": "getty.mapped.tsv",
    "getty_agree": "getty.agree.tsv",
    "correct_temp_1": "correct.temp.1.tsv",
    "correct_temp_2": "correct.temp.2.tsv",
    "incorrect_temp": "incorrect.temp.tsv",
    "correct": "correct.tsv",
    "incorrect": "incorrect.tsv",
    "unknown": "unknown.tsv"
}

for key, value in output_names.items():
    variable = key.upper()
    os.environ[variable] = os.path.join(output_path, value)
    kgtk_environment_variables.append(variable)

for variable in kgtk_environment_variables:
    print("{}: \"{}\"".format(variable, os.environ[variable]))

DATABASE: "/nas/home/bohuizha/KG/hunger-for-knowledge/data/"
WIKIDATA: "/nas/home/bohuizha/KG/hunger-for-knowledge/data/claims.tsv"
KGTK_LABEL_FILE: "/nas/home/bohuizha/KG/hunger-for-knowledge/data/labels.en.tsv"
P31: "/nas/home/bohuizha/KG/hunger-for-knowledge/data/P31.tsv"
P279STAR: "/nas/home/bohuizha/KG/hunger-for-knowledge/data/P279star.tsv"
ULAN: "/nas/home/bohuizha/KG/hunger-for-knowledge/data/gvp/ULAN/"
ULAN_GENDER_CONCAT_ID: "/nas/home/bohuizha/KG/hunger-for-knowledge/data/gvp/ULAN/ulan.gender.concat.id.tsv"
ULAN_ALIGN: "/nas/home/bohuizha/KG/hunger-for-knowledge/data/gvp/ULAN/wiki.align.tsv"
OUTPUT: "/nas/home/bohuizha/KG/hunger-for-knowledge/data/gvp/gender_output/"
WIKI_GENDER: "/nas/home/bohuizha/KG/hunger-for-knowledge/data/gvp/gender_output/wiki.gender.tsv"
WIKI_UNKNOWN: "/nas/home/bohuizha/KG/hunger-for-knowledge/data/gvp/gender_output/wiki.unknown.tsv"
PAIRS: "/nas/home/bohuizha/KG/hunger-for-knowledge/data/gvp/gender_output/pairs.tsv"
PATHS: "/nas/home/bohuizha/KG/hun

## Step 1: Query Wikidata
Query Wikidata for PoB for people that have an ULAN ID (92k people)

In [3]:
# count total people 
# ULAN ID
kgtk("""
    query -i $ULAN_ALIGN 
        --match '(ulanid)-[]->(qnode)'
        --return 'count(distinct ulanid)'
    """)

,"count(DISTINCT graph_627_c1.""node1"")"
0,92353


In [4]:
# Qnode
kgtk("""
    query -i $ULAN_ALIGN 
        --match '(ulanid)-[]->(qnode)'
        --return 'count(distinct qnode)'
    """)

,"count(DISTINCT graph_627_c1.""node2"")"
0,92210


In [4]:
%%time
kgtk("""
    query -i $ULAN_ALIGN $WIKIDATA
        --match 'w: (ulanid)-[]->(qnode), 
                 a: (qnode)-[:P21]->(gender)' 
        --return 'distinct ulanid as node1, "P21" as label, gender as node2'
        --limit 10
    / add-labels
    """)

CPU times: user 44.6 ms, sys: 26.7 ms, total: 71.3 ms
Wall time: 1min 23s


,node1,label,node2,label;label,node2;label
0,ulan:500023949,P21,Q6581097,'sex or gender'@en,'male'@en
1,ulan:500123827,P21,Q6581097,'sex or gender'@en,'male'@en
2,ulan:500223082,P21,Q6581097,'sex or gender'@en,'male'@en
3,ulan:500108173,P21,Q6581097,'sex or gender'@en,'male'@en
4,ulan:500072302,P21,Q6581072,'sex or gender'@en,'female'@en
5,ulan:500091345,P21,Q6581097,'sex or gender'@en,'male'@en
6,ulan:500059872,P21,Q6581097,'sex or gender'@en,'male'@en
7,ulan:500020713,P21,Q6581097,'sex or gender'@en,'male'@en
8,ulan:500160869,P21,Q6581097,'sex or gender'@en,'male'@en
9,ulan:500108146,P21,Q6581097,'sex or gender'@en,'male'@en


In [5]:
%%time
kgtk("""
    query -i $ULAN_ALIGN $WIKIDATA
        --match 'w: (ulanid)-[]->(qnode), 
                 a: (qnode)-[:P21]->(gender)' 
        --return 'distinct ulanid as node1, "P21" as label, gender as node2'
        -o $WIKI_GENDER
    """)

CPU times: user 21.1 ms, sys: 16.2 ms, total: 37.3 ms
Wall time: 35.7 s


## Step 2: Record Known and Unknown

In [6]:
# count known ULAN IDs
kgtk("""
    query -i $WIKI_GENDER
        --match '(ulanid)-[]->(gender)' 
        --return 'count(distinct ulanid)'
    """)

,"count(DISTINCT graph_654_c1.""node1"")"
0,89307


In [5]:
# count known Qnodes
kgtk("""
    query -i $WIKI_GENDER $ULAN_ALIGN
        --match 'g: (ulanid)-[]->(nation), a: (ulanid)-[]->(qnode)' 
        --return 'count(distinct qnode)'
    """)

,"count(DISTINCT graph_627_c2.""node2"")"
0,89188


Filter unknow in ULAN-Wikidata alignment

In [7]:
%%time
kgtk("""
    ifnotexists -i $ULAN_ALIGN 
        --filter-on $WIKI_GENDER 
        --input-keys node1 
        --filter-keys node1 
        -o $WIKI_UNKNOWN
    """)

CPU times: user 6.02 ms, sys: 8.92 ms, total: 14.9 ms
Wall time: 4.78 s


In [8]:
# count unknown ULAN IDs
kgtk("""
    query -i $WIKI_UNKNOWN
        --match '(ulanid)-[]->(qnode)'
        --return 'count(distinct ulanid)'
    """)

,"count(DISTINCT graph_655_c1.""node1"")"
0,3046


In [6]:
# count unknown Qnodes
kgtk("""
    query -i $WIKI_UNKNOWN
        --match '(ulanid)-[]->(qnode)'
        --return 'count(distinct qnode)'
    """)

,"count(DISTINCT graph_655_c1.""node2"")"
0,3022


## Step 3: Query Getty
Based on the known PoB for people, query Getty. Before we perform single property query (we can consider it as 1 hop paths query), this time, we generalize to paths between known pairs (Subjects, Objects). Similarily we will compute the distributions of found paths and pick the top 1 path (property chain). 

### 3-1: Build Pairs from Wikidata Results 
Pairs file is needed for performing kgtk paths operation as the query objects.

In [26]:
%%time
kgtk("""
    query -i $WIKI_GENDER
        --match '(ulanid)-[]->(gender)'
        --return 'distinct ulanid as source, gender as target'
        --limit 100
        -o $PAIRS
    """)

CPU times: user 14.6 ms, sys: 20.2 ms, total: 34.8 ms
Wall time: 6.19 s


In [27]:
!head $PAIRS | column -ts $'\t'

source          target
ulan:500023949  Q6581097
ulan:500123827  Q6581097
ulan:500223082  Q6581097
ulan:500108173  Q6581097
ulan:500072302  Q6581072
ulan:500091345  Q6581097
ulan:500059872  Q6581097
ulan:500020713  Q6581097
ulan:500160869  Q6581097


### 3-2: Perform Paths Mapping
This step is the core step of this query and it is also the bottleneck of running this procedure.

In [28]:
%%time
kgtk("""
    paths --max_hops 4
        --path-file $PAIRS
        --path-mode NONE 
        --path-source source
        --path-target target
        -i $ULAN_GENDER_CONCAT_ID
        --statistics-only
        -o $PATHS
    """)

CPU times: user 354 ms, sys: 211 ms, total: 565 ms
Wall time: 2min 47s


In [29]:
!head $PATHS

node1	label	node2	id
p0	0	ulan:500072302:foaf:focus:ulan:500072302-agent	p0-0-0
p0	1	ulan:500072302-agent:gvp:biographyPreferred:ulan_bio:4000127583	p0-1-1
p0	2	ulan_bio:4000127583:schema:gender:aat:300189557	p0-2-2
p0	3	aat:300189557:skos:exactMatch:Q6581072	p0-3-3
p1	0	ulan:500072302:foaf:focus:ulan:500072302-agent	p1-0-4
p1	1	ulan:500072302-agent:gvp:biographyNonPreferred:ulan_bio:4000798933	p1-1-5
p1	2	ulan_bio:4000798933:schema:gender:aat:300189557	p1-2-6
p1	3	aat:300189557:skos:exactMatch:Q6581072	p1-3-7
p2	0	ulan:500099612:foaf:focus:ulan:500099612-agent	p2-0-8


### 3-3: Select the Top 1 Property Chain
Here we use Pandas to sort the top 1 property chain. 

In [30]:
# load paths.tsv file
paths = pd.read_csv(os.environ['PATHS'], sep='\t')
# process property: since paths command generate id as node2, we need to further extract property from it
paths.node2 = paths.node2.apply(lambda x: ':'.join(x.split(':')[2:4]))
# save processed property
paths.to_csv(os.environ['PATHS_LABEL'], sep='\t', index=False)
# join property chain for each found path
paths_concat = paths.groupby(paths['node1']).agg({'node2': lambda x: ' '.join(list(x))})
# sort the top 
paths_concat.value_counts()#.head(1)

node2                                                             
foaf:focus gvp:biographyPreferred schema:gender skos:exactMatch       27
foaf:focus gvp:biographyNonPreferred schema:gender skos:exactMatch    15
dtype: int64

### 3-4: Query Getty Using the Property Chain
Query Getty for the values of the unknown people, with the found property chain: `(ulanid)-[foaf:focus]->()-[gvp:biographyPreferred]->()-[schema:gender]->()-[skos:exactMatch]->(Qnode of Gender)`

In [31]:
%%time
kgtk("""
    query -i $WIKI_UNKNOWN $ULAN_GENDER_CONCAT_ID
        --match 'unknown: (ulanid)-[]->(), 
                 ulan: (ulanid)-[p0]->()-[p1]->()-[p2]->()-[p3]->(gender)'
        --where 'p0.label = "foaf:focus" AND
                 p1.label = "gvp:biographyPreferred" AND 
                 p2.label = "schema:gender" AND
                 p3.label = "skos:exactMatch"'
        --return 'distinct ulanid as node1, "P21" as label, gender as node2'
        --limit 10
    / add-labels
    """)

CPU times: user 339 ms, sys: 162 ms, total: 501 ms
Wall time: 2min 27s


,node1,label,node2,label;label,node2;label
0,ulan:500000767,P21,Q44148,'sex or gender'@en,'male organism'@en
1,ulan:500001474,P21,Q43445,'sex or gender'@en,'female organism'@en
2,ulan:500001474,P21,Q6581072,'sex or gender'@en,'female'@en
3,ulan:500001678,P21,Q44148,'sex or gender'@en,'male organism'@en
4,ulan:500002563,P21,Q44148,'sex or gender'@en,'male organism'@en
5,ulan:500003826,P21,Q44148,'sex or gender'@en,'male organism'@en
6,ulan:500003921,P21,Q44148,'sex or gender'@en,'male organism'@en
7,ulan:500004056,P21,Q44148,'sex or gender'@en,'male organism'@en
8,ulan:500005174,P21,Q44148,'sex or gender'@en,'male organism'@en
9,ulan:500005840,P21,Q44148,'sex or gender'@en,'male organism'@en


In [34]:
%%time
kgtk("""
    query -i $WIKI_UNKNOWN $ULAN_GENDER_CONCAT_ID
        --match 'unknown: (ulanid)-[]->(), 
                 ulan: (ulanid)-[p0]->()-[p1]->()-[p2]->()-[p3]->(gender)'
        --where 'p0.label = "foaf:focus" AND
                 p1.label = "gvp:biographyPreferred" AND 
                 p2.label = "schema:gender" AND
                 p3.label = "skos:exactMatch"'
        --return 'distinct ulanid as node1, "P21" as label, gender as node2'
        -o $GETTY_GENDER
    """)

CPU times: user 9.15 ms, sys: 14.3 ms, total: 23.5 ms
Wall time: 4.47 s


Count how many new results (count in ULAN ID) we found:

In [35]:
kgtk("""
    query -i $GETTY_GENDER
        --match '(ulanid)-[]->(gender)'
        --return 'count(distinct ulanid)'
    """)

,"count(DISTINCT graph_657_c1.""node1"")"
0,496


## Step 4: Record New Results

Map back to Wikidata

In [36]:
%%time
kgtk("""
    query -i $GETTY_GENDER $ULAN_ALIGN
        --match 'g: (ulanid)-[p]->(gender), 
                 w: (ulanid)-[]->(qnode)'
        --return 'distinct qnode as node1, p.label as label, gender as node2'
        -o $GETTY_MAPPED
    """)

CPU times: user 18 ms, sys: 11.2 ms, total: 29.2 ms
Wall time: 4.23 s


Count how many new results (count in Qnode) we found

In [37]:
kgtk("""
    query -i $GETTY_MAPPED
        --match '(qnode)-[]->(gender)'
        --return 'count(distinct qnode)'
    """)

,"count(DISTINCT graph_658_c1.""node1"")"
0,495


## Step 5: Validate with Wikidata Constraints
Property `P21` doesn't have value-type constraints. Here since the range of sex or gender is not that wide, we can check the distinct values appear in our results.

In [43]:
kgtk("""
    query -i $GETTY_MAPPED
        --match '(qnode)-[]->(gender)'
        --return 'distinct gender'
    / add-labels
    """)

,node2,node2;label
0,Q44148,'male organism'@en
1,Q43445,'female organism'@en
2,Q6581072,'female'@en


## Step 6: Record Unknown after Query
We record unknowns after query Getty.

In [39]:
kgtk("""
    ifnotexists -i $WIKI_UNKNOWN 
        --filter-on $GETTY_GENDER 
        --input-keys node1 
        --filter-keys node1
        -o $UNKNOWN
    """)

In [40]:
# count in ULAN IDs
kgtk("""
    query -i $UNKNOWN
        --match '(ulanid)-[]->(qnode)'
        --return 'count(distinct ulanid)'
    """)

,"count(DISTINCT graph_659_c1.""node1"")"
0,2550


In [7]:
# count in Qnodes
kgtk("""
    query -i $UNKNOWN
        --match '(ulanid)-[]->(qnode)'
        --return 'count(distinct qnode)'
    """)

,"count(DISTINCT graph_659_c1.""node2"")"
0,2528


# Step 7: Comparison of the Known Values
How often do the results from two knowledge graphs agree

In [4]:
%%time
kgtk("""
    query -i $ULAN_ALIGN $ULAN_GENDER_CONCAT_ID
        --match 'w: (ulanid)-[]->(), 
                 u: (ulanid)-[p0]->()-[p1]->()-[p2]->()-[p3]->(gender)'
        --where 'p0.label = "foaf:focus" AND
                 p1.label = "gvp:biographyPreferred" AND 
                 p2.label = "schema:gender" AND
                 p3.label = "skos:exactMatch"'
        --return 'distinct ulanid as node1, "P21" as label, gender as node2'
        -o $GETTY_AGREE
    """)

CPU times: user 5.75 ms, sys: 12.4 ms, total: 18.1 ms
Wall time: 7.07 s


Count agree in Qnode:

In [26]:
%%time
kgtk("""
    query -i $WIKI_GENDER $GETTY_AGREE $ULAN_ALIGN
        --match 'w: (ulanid)-[]->(gender), g: (ulanid)-[]->(gender), a: (ulanid)-[]->(qnode)'
        --return 'count(distinct qnode)'
    """)

CPU times: user 10.4 ms, sys: 12.5 ms, total: 22.9 ms
Wall time: 6.2 s


,"count(DISTINCT graph_627_c3.""node2"")"
0,8814


Count value per node:

In [22]:
nodes = kgtk("""
    query -i $GETTY_AGREE $ULAN_ALIGN 
        --match 'g: (ulanid)-[]->(), w: (ulanid)-[]->(qnode)'
        --return 'count(distinct qnode) as node'
    """)
nodes

,node
0,85248


In [23]:
import subprocess

values = subprocess.check_output("wc -l < $GETTY_AGREE", shell=True)
values = values.decode("utf-8").strip()
values = int(values) - 1

In [24]:
values / nodes.iloc[0]['node']

1.1068177552552552